# Clean and Pre-process BPIC'2011 Dataset 

## Dataset description

The raw dataset includes

## Import library

In [1]:
import os
import pandas as pd
import json
!pip install natsort
from natsort import natsorted
import numpy as np
# from collections import Counter
from datetime import datetime
from dateutil.relativedelta import relativedelta

## Set Working Folders

In [2]:
folder = os.getcwd()
raw_path = os.path.join(folder, "raw_data")
processed_path = os.path.join(folder, "processed_data")


## Read dataset from CSV file

In [3]:
csv_file = os.path.join(raw_path, "Hospital log.csv")

raw_data = pd.read_csv(csv_file, sep=";", low_memory=False)

# show all columns
pd.set_option('display.max_columns', None)

df = raw_data
df.shape

(150291, 128)

## Data cleaning process 

...

### Step 1: Deleting irrelevant attributes

- Attributes for traces 
    - case:concept:name 
    - Age, Age:1-15
    - ~~Diagnosis, Diagnosis:1-15~~
    - Diagnosis code, Diagnosis code:1-15
    - Treatment code, Treatment code:1-15
    - Diagnosis Treatment Combination ID, Diagnosis Treatment Combination ID:1-15
    - ~~Specialism code, Specialism code:1-15~~
    - Start date, Start date:1-15
    - End date, End date:1-15


- Attributes for events
    - org:group
    - Number of executions
    - ~~Specialism code~~
    - event:concept:name
    - Producer code
    - Section
    - Activity code
    - time:timestamp
    - ~~lifecycle:transition~~

In [4]:
# Clean all columns with "Diagnosis Treatment Combination ID", "Specialism code" and "lifecycle:transition"
df = df.loc[:,~df.columns.str.startswith('Diagnosis:')]
df = df.drop(columns='Diagnosis')
df = df.loc[:,~df.columns.str.startswith('Specialism code')] 
df = df.loc[:,~df.columns.str.startswith('lifecycle:transition')]
df = df.drop(columns='Unnamed: 127')
df.shape


(150291, 94)

### Step 2: Combining repeating features to a list

Cleaning method from IEEE-XAI paper:

In [5]:
# Sort all columns in alphabetical order

df = df.reindex(natsorted(df.columns), axis=1)
df.head(1)

,Activity code,Age,Age:1,Age:2,Age:3,Age:4,Age:5,Diagnosis Treatment Combination ID,Diagnosis Treatment Combination ID:1,Diagnosis Treatment Combination ID:2,Diagnosis Treatment Combination ID:3,Diagnosis Treatment Combination ID:4,Diagnosis Treatment Combination ID:5,Diagnosis Treatment Combination ID:6,Diagnosis Treatment Combination ID:7,Diagnosis Treatment Combination ID:8,Diagnosis Treatment Combination ID:9,Diagnosis Treatment Combination ID:10,Diagnosis Treatment Combination ID:11,Diagnosis Treatment Combination ID:12,Diagnosis Treatment Combination ID:13,Diagnosis Treatment Combination ID:14,Diagnosis Treatment Combination ID:15,Diagnosis code,Diagnosis code:1,Diagnosis code:2,Diagnosis code:3,Diagnosis code:4,Diagnosis code:5,Diagnosis code:6,Diagnosis code:7,Diagnosis code:8,Diagnosis code:9,Diagnosis code:10,Diagnosis code:11,Diagnosis code:12,Diagnosis code:13,Diagnosis code:14,Diagnosis code:15,End date,End date:1,End date:2,End date:3,End date:4,End date:5,End date:6,End date:7,End date:8,End date:9,End date:10,End date:11,End date:12,End date:13,End date:14,End date:15,Number of executions,Producer code,Section,Start date,Start date:1,Start date:2,Start date:3,Start date:4,Start date:5,Start date:6,Start date:7,Start date:8,Start date:9,Start date:10,Start date:11,Start date:12,Start date:13,Start date:14,Start date:15,Treatment code,Treatment code:1,Treatment code:2,Treatment code:3,Treatment code:4,Treatment code:5,Treatment code:6,Treatment code:7,Treatment code:8,Treatment code:9,Treatment code:10,Treatment code:11,Treatment code:12,Treatment code:13,Treatment code:14,Treatment code:15,case:concept:name,event:concept:name,org:group,time:timestamp
0,410100,33.0,NaN,NaN,NaN,NaN,NaN,181229.0,376907.0,376908.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M13,106,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-04T23:45:36+01:00,2005-01-29T23:45:36+01:00,2005-01-29T23:45:36+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,SRTH,Section 5,2005-01-05T00:14:24+01:00,2005-01-03T00:14:24+01:00,2005-01-03T00:14:24+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.0,13.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1e consult poliklinisch,Radiotherapy,2005-01-03T00:00:00+01:00


In [6]:
# function for finding last valid value
def findLastValid(x):
    list_of_last_value = []
    for index, row in x.iterrows():
        if row.last_valid_index() is None:
            list_of_last_value.append(np.nan)
        else:
            list_of_last_value.append(row[row.last_valid_index()])
    
    return list_of_last_value

# function for finding the valid value list
def findValidList(x):
    list_of_values = []
    for index, row in x.iterrows():
        values = []
        for i, v in row.items():
            if pd.notnull(v):
                values.append(v)
        list_of_values.append(values)
    
    return list_of_values

# function for finding most frequent element
def mostFrequent(x):
    list_of_values=[]
    for case in x:
        if len(case) == 0:
            list_of_values.append(None)
        else:
            list_of_values.append(max(set(case), key = case.count))
    return list_of_values


In [7]:
# latest Age, Diagnosis Code, Treatment code, Diagnosis Treatment Combination ID
Age_Latest = findLastValid(df.loc[:, df.columns.str.startswith('Age')])
# DiagnosisCode_Latest = findLastValid(df.loc[:, df.columns.str.startswith('Diagnosis code')])
# TreatmentCode_Latest = findLastValid(df.loc[:, df.columns.str.startswith('Treatment code')])
# CombinationID_Latest = findLastValid(df.loc[:, df.columns.str.startswith('Diagnosis Treatment Combination ID')])

# Age_List = findValidList(df.loc[:, df.columns.str.startswith('Age')])
DiagnosisCode_List = findValidList(df.loc[:, df.columns.str.startswith('Diagnosis code')])
DiagnosisCode_List = mostFrequent(DiagnosisCode_List)

TreatmentCode_List = findValidList(df.loc[:, df.columns.str.startswith('Treatment code')])
TreatmentCode_List = mostFrequent(TreatmentCode_List)

CombinationID_List = findValidList(df.loc[:, df.columns.str.startswith('Diagnosis Treatment Combination ID')])
CombinationID_List = mostFrequent(CombinationID_List)

In [8]:
# Calculate year

# total number of traces: 1143

Year_list = []

for case_id in range(1143):
    year_df = df.loc[df['case:concept:name'] == case_id]
    dateStart = datetime.strptime(year_df.iloc[0,-1], '%Y-%m-%dT%H:%M:%S%z')
    dateEnd = datetime.strptime(year_df.iloc[-1,-1], '%Y-%m-%dT%H:%M:%S%z')
    Year_list.append(relativedelta(dateEnd, dateStart).years)



The new dataframe should include these feaures: 

Activity, Department, Number of executions, Activitycode, Producer code, Section, Age, Diagnosis Code, Treatment code, Diagnosis Treatment Combination ID and Year.

In [9]:
processed_df = df[['case:concept:name', 'event:concept:name', 'org:group', 'time:timestamp', 'Activity code', 'Number of executions', 'Producer code', 'Section']]

In [10]:
processed_df = processed_df.assign(Age = Age_Latest)
processed_df = processed_df.assign(Diagnosis_code = DiagnosisCode_List)
processed_df = processed_df.assign(Treatment_code = TreatmentCode_List)
processed_df = processed_df.assign(CombinationID = CombinationID_List)

In [11]:
def input_year(row):
    return Year_list[row['case:concept:name']]

processed_df['Total year'] = processed_df.apply(lambda x: input_year(x), axis=1)


In [12]:
processed_df.head()

,case:concept:name,event:concept:name,org:group,time:timestamp,Activity code,Number of executions,Producer code,Section,Age,Diagnosis_code,Treatment_code,CombinationID,Total year
0,0,1e consult poliklinisch,Radiotherapy,2005-01-03T00:00:00+01:00,410100,1,SRTH,Section 5,33.0,106,23.0,376907.0,0
1,0,administratief tarief - eerste pol,Radiotherapy,2005-01-03T00:00:00+01:00,419100,1,SRTH,Section 5,33.0,106,23.0,376907.0,0
2,0,verlosk.-gynaec. korte kaart kosten-out,Nursing ward,2005-01-05T00:00:00+01:00,10107,1,SGEH,Section 2,33.0,106,23.0,376907.0,0
3,0,echografie - genitalia interna,Obstetrics & Gynaecology clinic,2005-01-05T00:00:00+01:00,339486E,1,SGEC,Section 2,33.0,106,23.0,376907.0,0
4,0,1e consult poliklinisch,Nursing ward,2005-01-05T00:00:00+01:00,410100,1,SGEH,Section 2,33.0,106,23.0,376907.0,0


### Step 3: Clean errors and missing values


In [13]:
clean_df = processed_df

clean_df = clean_df[clean_df['Number of executions'] > 0]

activity_list = clean_df['event:concept:name'].value_counts().loc[lambda x : x>19].index.tolist()
clean_df = clean_df[clean_df['event:concept:name'].isin(activity_list)]



In [14]:
# clean_df.isnull().sum().sum()
clean_df.isnull().any()

case:concept:name       False
event:concept:name      False
org:group               False
time:timestamp          False
Activity code           False
Number of executions    False
Producer code           False
Section                 False
Age                     False
Diagnosis_code          False
Treatment_code           True
CombinationID           False
Total year              False
dtype: bool

In [15]:
filter_nan_treatment = clean_df[~clean_df['Treatment_code'].isnull()]
filter_nan_treatment.isnull().any()
# yes_nan = clean_df[~clean_df['Treatment_code'].isnull()]

case:concept:name       False
event:concept:name      False
org:group               False
time:timestamp          False
Activity code           False
Number of executions    False
Producer code           False
Section                 False
Age                     False
Diagnosis_code          False
Treatment_code          False
CombinationID           False
Total year              False
dtype: bool

In [16]:
all_nan = clean_df[clean_df['Treatment_code'].isnull()]

In [17]:
replace_nan_treatment = clean_df
replace_nan_treatment.isnull().any()

case:concept:name       False
event:concept:name      False
org:group               False
time:timestamp          False
Activity code           False
Number of executions    False
Producer code           False
Section                 False
Age                     False
Diagnosis_code          False
Treatment_code           True
CombinationID           False
Total year              False
dtype: bool

In [18]:

diag_and_treat = filter_nan_treatment[['Diagnosis_code','Treatment_code']]

for index, row in all_nan.iterrows():
    diag = row['Diagnosis_code']
    
    filterdf = diag_and_treat[(diag_and_treat['Diagnosis_code'] == diag)]
    
    replace_nan_treatment.loc[index, ['Treatment_code']] = filterdf['Treatment_code'].value_counts().index.tolist()[0]
    

In [19]:
replace_nan_treatment.isnull().any()

case:concept:name       False
event:concept:name      False
org:group               False
time:timestamp          False
Activity code           False
Number of executions    False
Producer code           False
Section                 False
Age                     False
Diagnosis_code          False
Treatment_code          False
CombinationID           False
Total year              False
dtype: bool

In [20]:
print(replace_nan_treatment["case:concept:name"].nunique(), filter_nan_treatment['case:concept:name'].nunique())

1142 1130


### Step 4: Output

In [21]:
column_names = ['CaseID', 'Activity', 'Department', 'Timestamps', 'Activity code', 
                'Number of executions', 'Producer code', 'Section', 'Age', 'Diagnosis code', 
                'Treatment code', 'Diagnosis Treatment Combination ID', 'Year']
replace_nan_treatment.columns = column_names
filter_nan_treatment.columns = column_names


In [25]:

output_csv = os.path.join(processed_path, "detele_nan_treatment (1130 cases).csv")
filter_nan_treatment.to_csv(output_csv)

